In [1]:
from GarbageDetector import GarbageDetector 
from ultralytics import YOLO
from config import test_config
import cv2
from BatchGarbageTester import BatchGarbageTester

## 单张图片测试

自定义检测模块

In [2]:
detector = GarbageDetector(model_path=test_config['model_path'])

In [3]:
results = detector.detect(image_input=test_config['img_path'])


image 1/1 d:\Desktop\\export_oi\images\test\0ec4b1c27046c4ca.jpg: 640x480 2 Bottles, 382.3ms
Speed: 19.6ms preprocess, 382.3ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 480)


In [4]:
for detection in results['detections']:
    print(f"找到 {detection['class']}, 置信度: {detection['confidence']:.2f}")

找到 瓶子, 置信度: 0.90
找到 瓶子, 置信度: 0.89


或者直接调用模型即可

In [5]:
model = YOLO(test_config['model_path'])

In [6]:
img = cv2.imread(test_config['img_path'])
result_single = model(img, conf=0.64)


0: 640x480 2 Bottles, 332.8ms
Speed: 5.2ms preprocess, 332.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 480)


In [7]:
for r in result_single:
    boxes = r.boxes  # 获取检测框信息（坐标、置信度、类别）
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # 检测框坐标（左上角/右下角）
        conf = box.conf[0].cpu().numpy()  # 置信度
        cls = int(box.cls[0].cpu().numpy())  # 类别ID
        cls_name = model.names[cls]  # 类别名称
        
        # 绘制检测框（可选）
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, f"{cls_name} {conf:.2f}", (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [8]:
# cv2.imshow("YOLO Result", img)
# cv2.waitKey(0)

In [9]:
# model.val()

## 批量图片测试

In [10]:
def run_comprehensive_test():
    """运行全面的批量测试"""
    
    # 初始化测试器
    tester = BatchGarbageTester(
        model_path=test_config['model_path'],
        test_images_dir=test_config['batch_imgs_path'],  # 包含1000张测试图片的目录
        output_dir=test_config['test_output_dir']
    )
    
    # 运行批量测试
    results = tester.run_batch_test(
        confidence_threshold=0.64,
        iou_threshold=0.5,
    )
    
    # 获取摘要
    summary = tester.get_summary()
    print("\n📋 测试摘要:")
    print(f"总测试图片: {summary['total_images']}")
    print(f"总检测数量: {summary['total_detections']}")
    print(f"平均推理时间: {summary['avg_inference_time']:.3f}秒")
    print(f"平均置信度: {summary['avg_confidence']:.3f}")
    print("分类统计:", summary['category_breakdown'])
    
    return tester

In [11]:
run_comprehensive_test()

🚀 开始批量测试...


测试进度:  29%|██▉       | 290/1000 [02:12<05:59,  1.97it/s, 已检测=290, 平均时间=0.454s]